# データ分析や機械学習における日本語の変数名の取り扱い

ビジネス関連のデータセットの列名が日本語になっていますよね。
どのようにして日本語ができないチームメンバーと、**効率的**に連携しながら
データ分析業務を行っていきますか。
**効率的**とは、お互いのソースコードを読めたり、
変数名(以下、特徴量名、列名ともいう）について意思疎通ができたりする
ということです。

> 取り上げている「日本語」はあくまでも、プロジェクトメンバー全員が精通しているわけではない、とある自然言語を象徴しているのに過ぎません。
> 中国語や韓国語などに読み替えても問題ありません。

東京 2021/11/23, Vo Chi Cong

国際的なデータ分析・機械学習チームに所属しながら、業務で当然のように日本語の変数名を使っていませんか。
お客様が日本企業ならば、データセットの列名が日本語になっていることがほとんどです。
これらの列名がそのまま特徴量の名前、いわゆる説明変数の名前として扱われることが多く見られます。
このままだと、専門性の高い外国人データアナリスト、データエンジニア、データサイエンティストの
プロジェクト参画の障害になりかねません。

また、お客様企業から更新データが提供されるたびに、または他システムからデータが連携されてくるたびに、
列名が時に変わっていませんか。どうしたら、前処理の工程で列名変更を吸収し、
後続の分析・学習・予測行程に影響が及ばないようにできるのでしょうか。

最後に、納品や報告用の資料にはどうしたら、正確な特徴量名称を適用することができるのでしょうか。
この正確性は、プロジェクトでの取り決めによるもので、プロジェクトの進行中に特徴量の名称が
幾度変更されても、最終的には、最新のルールに従って表示する必要があります。

# 日本語変数名の問題点

日本語変数名をそのまま直接使う場合に生じうる問題点を具体的に見ていきましょう。

簡単な例として、「名」と「歳」とい2つだけの列からなる個人情報テーブル（西遊記の中心人物)
`df_in` があるとします。

In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "last_expr_or_assign"
import pandas as pd


In [2]:
df_in = pd.DataFrame(dict(名=["孫 悟空", "猪 八戒", "沙 悟浄", "玄奘 三蔵"], 歳=[22, 19, 18, 30]))


,名,歳
0,孫 悟空,22
1,猪 八戒,19
2,沙 悟浄,18
3,玄奘 三蔵,30


わくわくしてすぐにでもデータ分析(EDA)に取り掛かる衝動に駆られていませんか。
しかしそうすると、「名」や「歳」（実案件では、沢山の日本語列名）を含んだソースコードが瞬く間に急速に記述されていきます。
私は、うんざりするほど何度も日本語の特徴量名が大量に散りばめられたJupyterノートブックを目にしてきました。
何を隠そう、自分自身もそのようなノートブックの作者の一人でした。

しかし、問題はすぐに顕在化してきます。

1. 日本語の変数名が点在するソースコードは、見苦しく、読み解きづらい。まして、日本語ができないメンバーには、実に読めない。
Google翻訳を使って読みなさいという馬鹿なアドバイスも遠くからかすかに聞こえてきそうですが、
変数名を翻訳しながらプログラムを読むことはあり得ないのでパスです。
2. まともにプログラムを書く人、またはその段階になった時の人は、キーボードで日本語変換をオンにして変数を呼び出すことを嫌がり、
基本的に英語（英数字）でプログラムを書きたがります。なお、各種開発環境やエディタのコード補完機能は、基本的に英数字以外の変数名はサポートしないはずです。
3. 日本語の曖昧性は、特徴量名に現れることがあります。冗長な特徴量名が付けられることもあります。
すなわち、もともと良くない特徴量名が付けられることがあって、日本語ができる人でも何の特徴量なのか
簡単に理解できません。

# 英語名で正規化

上記の問題を回避するために、前処理の一ステップとして、まずは `names_j2e` というようなマッピングを用意して、日本語の列名を一気に英語に変換しましょう。

1. このマッピングはプロジェクトで統一的に利用する。特徴量の本質が変わらない限り、英語の列名は原則変更しない。
これらの英語名は、データ分析、学習、予測の行程を通して一貫して利用する。
2. 表記の揺らぎを認め、複数の日本語名称を1つの英語名称にマッピングするケースもある。
プロジェクト進行中に、日本語の列名が変わることもあるし、複数のテーブルにおいて、ある特徴量が複数の名称で参照されていることも少なくない。
そのようなケースでも、1つだけの英語名称で正規化する。

In [3]:
from collections import OrderedDict

names_j2e = OrderedDict(
    フルネーム="full_name",
    名="full_name",
    氏名="full_name",
    歳="age",
    年齢="age",
)


OrderedDict([('フルネーム', 'full_name'),
             ('名', 'full_name'),
             ('氏名', 'full_name'),
             ('歳', 'age'),
             ('年齢', 'age')])

In [4]:
def rename_j2e(df: pd.DataFrame) -> pd.DataFrame:
    return df.rename(columns=names_j2e)


前処理で `df_work` というような、英語列名のデータフレームを作り、これでデータ分析、機械学習、予測処理を行っていくわけです。
列名が英語になったので、多国籍メンバの間で会話が捗り、コーディングもスピードアップします。
日本語の名称が変わっても、前処理のステップでマッピングの変更によって吸収できるので、
重要な後続行程に影響を及ぼさないで済みます。

In [5]:
df_work = rename_j2e(df_in)


,full_name,age
0,孫 悟空,22
1,猪 八戒,19
2,沙 悟浄,18
3,玄奘 三蔵,30


# 日本語の変数名でレポーティング

やはり、日本のお客様に対しては、報告書における変数名は日本語でないといけません。
更に、その変数名は、プロジェクトの最新ルールに従う必要があります。
つまり、たとえ入力データセットの列名に揺らぎがあったり、間違いがあったりしても、
出力レポートにおいては、最後に決められた正しい名称表記を
柔軟に適用しなければなりません。

プロジェクトチームが使っている変数の英語名称を正確にレポート用の日本語に変換するためには、
`names_e2j` というような変換規則が必要です。
このマッピングは、 前述の `names_j2e` から割り出すのですが、
最終的に正しい日本語名称へのマッピング情報だけを保持します。

In [6]:
names_e2j = OrderedDict({v: k for k, v in names_j2e.items()})


OrderedDict([('full_name', '氏名'), ('age', '年齢')])

In [7]:
def rename_e2j(df: pd.DataFrame) -> pd.DataFrame:
    return df.rename(columns=names_e2j)


In [8]:
df_out = rename_e2j(df_work)


,氏名,年齢
0,孫 悟空,22
1,猪 八戒,19
2,沙 悟浄,18
3,玄奘 三蔵,30


# まとめ

国際的なデータ分析・機械学習チームで日本語の列名を扱う段取りを次のように提案します。

1. 前処理で、日本語の列名を英語に変換し、揺らぎを吸収し、統一的に正規化します。
2. データ分析、学習、予測では、英語の列名を一貫性をもって利用します。
3. 報告、納品用のデータは、作業成果データの列名を英語から正規な日本語名称に変換します。

参考リンク

- [Tên biến tiếng Nhật trong phân tích dữ liệu và học máy](t.ly/AgAv) (ベトナム語)
- Julia と Python の [sample code](https://github.com/vochicong/dataframe-i18n)